In [1]:
!pip install biopython

In [31]:
#!pip install tensorflow

In [5]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from Bio.SeqIO import parse
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from sklearn.preprocessing import LabelEncoder
import re
import math
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv1D, MaxPooling1D
from collections import Counter
from sklearn.utils import class_weight
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()
df = pd.read_csv(io.StringIO(uploaded['final_dates_reduced.csv'].decode('utf-8')))
df.head(2)

Saving final_dates_reduced.csv to final_dates_reduced (1).csv


,Accession,Release_Date,Species,Length,Geo_Location,Host,Isolation_Source,Collection_Date,Sequence
0,MT683386,2020-07-01T00:00:00Z,Severe acute respiratory syndrome-related coro...,29858,USA,Homo sapiens,NaN,Apr-May,GGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTT...
1,MT683387,2020-07-01T00:00:00Z,Severe acute respiratory syndrome-related coro...,29854,USA,Homo sapiens,NaN,Apr-May,ATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGA...


In [6]:
!pip install keras-metrics

In [7]:
df3 = df.sample(frac=1).reset_index(drop=True)
df3.head(5)

,Accession,Release_Date,Species,Length,Geo_Location,Host,Isolation_Source,Collection_Date,Sequence
0,MT246463,2020-03-26T00:00:00Z,Severe acute respiratory syndrome-related coro...,29782,USA: WA,Homo sapiens,NaN,Mar,GTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACT...
1,MT585050,2020-06-10T00:00:00Z,Severe acute respiratory syndrome-related coro...,29826,USA: Michigan,Homo sapiens,swab,Apr-May,ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
2,MT642381,2020-06-19T00:00:00Z,Severe acute respiratory syndrome-related coro...,29858,"USA: Washington, Pierce County",Homo sapiens,NaN,Apr-May,TACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGAT...
3,MT252688,2020-04-24T00:00:00Z,Severe acute respiratory syndrome-related coro...,29864,USA: WA,Homo sapiens,NaN,Mar,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
4,MT446355,2020-05-08T00:00:00Z,Severe acute respiratory syndrome-related coro...,29807,"USA: Slidell, LA",Homo sapiens,oronasopharynx,Apr-May,GTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGC...


In [8]:
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

In [9]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

LabelEncoder()

In [10]:
def ordinal_encoder(my_array):
    
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    
    return float_encoded

In [11]:
data=df3[["Sequence","Collection_Date"]]
data=data[data["Sequence"].notna()]

dummy=[]
dum=np.array(dummy)

form={"inp_seq":dum}

seq_df = pd.DataFrame (form, columns = ['inp_seq'])

seq_list=[]

for idx, seq in enumerate(list(data["Sequence"])):
    arr=ordinal_encoder(string_to_array(seq))
    seq_list.append(arr)
    
seq_df["inp_seq"]=seq_list

final_data= data.assign(enc_seq=seq_df)

final_data=final_data[["enc_seq","Collection_Date"]]

final_data

,enc_seq,Collection_Date
0,"[0.75, 1.0, 0.25, 1.0, 0.25, 0.25, 1.0, 1.0, 0...",Mar
1,"[0.25, 0.5, 1.0, 1.0, 1.0, 0.5, 0.75, 0.25, 1....",Apr-May
2,"[1.0, 0.25, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5,...",Apr-May
3,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Mar
4,"[0.75, 1.0, 1.0, 0.5, 1.0, 0.5, 1.0, 0.25, 0.2...",Apr-May
...,...,...
5592,"[0.75, 0.75, 1.0, 1.0, 1.0, 0.25, 1.0, 0.25, 0...",Apr-May
5593,"[1.0, 0.5, 0.5, 0.5, 0.25, 0.75, 0.75, 1.0, 0....",Apr-May
5594,"[1.0, 0.25, 0.25, 0.5, 0.25, 0.25, 0.25, 0.5, ...",Apr-May
5595,"[1.0, 0.5, 1.0, 0.5, 1.0, 0.25, 0.25, 0.25, 0....",Mar


In [12]:
def get_maxLen(enc_seq):
    
    max=0
    for row in enc_seq:
        #print(type(row))
        if(len(row)>max):
            max=len(row)
    
    return max

In [13]:
def append_arr(enc_seq, max_len):
    
    seq_l=list(enc_seq)
    for i in range(len(seq_l),max_len):
        seq_l.append(0)
        
    new_seq_ar=np.array(seq_l)

        
    return new_seq_ar

In [14]:
import math

max_len=get_maxLen(final_data["enc_seq"])
# print("max_len is",max_len)
# if max_len%2 !=0:
#     max_len += 1
max_len = 29929

n = 1
for i in range(1, math.floor(math.sqrt(max_len))):
    if max_len%i==0:
        n = i

padded_seq_list=[]

for index, row in final_data.iterrows():
    seq_ar=append_arr(row["enc_seq"],max_len)
    padded_seq_list.append(seq_ar)

dummy=[]
dum=np.array(dummy)
form={"padded_enc_seq":dum}
padded_seq_df = pd.DataFrame (form, columns = ['padded_enc_seq'])

padded_seq_df["padded_enc_seq"]=padded_seq_list

padded_final_data= final_data.assign(padded_enc_seq=padded_seq_df)

padded_final_data=padded_final_data[["padded_enc_seq","Collection_Date"]]

padded_final_data.head(5)

,padded_enc_seq,Collection_Date
0,"[0.75, 1.0, 0.25, 1.0, 0.25, 0.25, 1.0, 1.0, 0...",Mar
1,"[0.25, 0.5, 1.0, 1.0, 1.0, 0.5, 0.75, 0.25, 1....",Apr-May
2,"[1.0, 0.25, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5,...",Apr-May
3,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Mar
4,"[0.75, 1.0, 1.0, 0.5, 1.0, 0.5, 1.0, 0.25, 0.2...",Apr-May


In [15]:
def reshape_seq(seq, m ,n):
    
    r_seq=np.reshape(seq,(m,n))
    
    return r_seq

re_seqList=[]

for index, row in padded_final_data.iterrows():
    #seq_ar=reshape_seq(row["padded_enc_seq"],n,max_len//n)
    seq_ar=reshape_seq(row["padded_enc_seq"],173, 173)
    re_seqList.append(seq_ar)

In [16]:
X = np.asarray(re_seqList)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))
X.shape

(5597, 173, 173, 1)

In [17]:
Y = padded_final_data['Collection_Date'].values
Y.shape

(5597,)

In [18]:
loc_classes = list(Y)
loc_classes = np.array(loc_classes) 
loc_classes = list(np.unique(loc_classes))

In [19]:
for i, x in enumerate(loc_classes):
    print(i, x)

0 Apr-May
1 Jan-Feb
2 Mar


In [20]:
df2 = padded_final_data['Collection_Date'].apply(loc_classes.index)
Y = df2.values
Y.shape

(5597,)

In [21]:
#x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.30)

In [22]:
model_best = Sequential()

In [23]:
class CheckMetrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_accs = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, Y_val = self.validation_data[0],self.validation_data[1]
                
        Y_val = np.argmax(Y_val,axis=1)
        
        Y_pred = self.model.predict(X_val)
        Y_pred = np.argmax(Y_pred,axis=1)

        _val_acc = accuracy_score(Y_val, Y_pred)

        self.val_accs.append(_val_acc)
        
        if _val_acc == max(self.val_accs):
            print("Validation Accuracy has improved. Saving Model.")
            self.model.save('modelCNN.h5')
            model_best = self.model

        return

In [24]:
keras_callbacks = CheckMetrics()

K-Fold Cross Validation

In [25]:
kfold = StratifiedKFold(n_splits=3, shuffle=True)
cvscores = []
for train, test in kfold.split(X, Y):
    batch_size = 32 
    num_classes = 3
    epochs = 29
    img_rows, img_cols = X[train].shape[1], X[train].shape[2]
    # X_train = X[train].reshape(X[train].shape[0],img_rows,img_cols,1)
    # X_test = X[test].reshape(X[test].shape[0],img_rows,img_cols,1)
    # convert class vectors to binary class matrices
    Y_train = keras.utils.to_categorical(Y[train], num_classes)
    Y_test = keras.utils.to_categorical(Y[test], num_classes)

    # create model
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
  
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
  
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
  
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

	  # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

    # Fit the model
    history = model.fit(X[train], Y_train, batch_size=batch_size, epochs=300, verbose=1, validation_data=(X[test], Y_test), callbacks=[keras_callbacks])
 
	  # evaluate the model
    model_sel = load_model('modelCNN.h5')
    scores = model_sel.evaluate(X[test], Y_test, verbose=0)
    #scores = model.evaluate(X[test], Y_test, verbose=0)
    print("%s: %.2f%%" % (model_sel.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

Train on 3731 samples, validate on 1866 samples
Epoch 1/300
3731/3731 [==============================] - 19s 5ms/step - loss: 0.8171 - accuracy: 0.5331 - val_loss: 0.7740 - val_accuracy: 0.5509
Validation Accuracy has improved. Saving Model.
Epoch 2/300
3731/3731 [==============================] - 10s 3ms/step - loss: 0.7843 - accuracy: 0.5403 - val_loss: 0.7763 - val_accuracy: 0.5509
Validation Accuracy has improved. Saving Model.
Epoch 3/300
3731/3731 [==============================] - 10s 3ms/step - loss: 0.7818 - accuracy: 0.5473 - val_loss: 0.7778 - val_accuracy: 0.5509
Validation Accuracy has improved. Saving Model.
Epoch 4/300
3731/3731 [==============================] - 10s 3ms/step - loss: 0.7819 - accuracy: 0.5481 - val_loss: 0.7790 - val_accuracy: 0.5509
Validation Accuracy has improved. Saving Model.
Epoch 5/300
3731/3731 [==============================] - 10s 3ms/step - loss: 0.7812 - accuracy: 0.5460 - val_loss: 0.7741 - val_accuracy: 0.5509
Validation Accuracy has improv

In [28]:
for i in range(0, len(cvscores)):
  print(f'> Fold {i+1}: Validation Accuracy = {cvscores[i]}%')

print(f'\nAverage Validation Accuracy: {np.mean(cvscores)}')
print(f'Variance: {np.std(cvscores)}')

> Fold 1: Validation Accuracy = 72.07931280136108%
> Fold 2: Validation Accuracy = 73.04394245147705%
> Fold 3: Validation Accuracy = 72.22520112991333%

Average Validation Accuracy: 72.44948546091716
Variance: 0.42454303772494656


In [29]:
# Y_pred = best_model.predict(x_val)
# y_pred = np.argmax(Y_pred, axis=1)

# print('Confusion Matrix - Test')
# y_val2 = np.argmax(y_val, axis=1)

# print(confusion_matrix(y_val2, y_pred))
# print('Classification Report')
# target_names = ['Apr-May', 'Jan-Feb', 'Mar']
# print(classification_report(y_val2, y_pred, target_names=target_names))

In [30]:
# Y_pred2 = best_model.predict(x_train)
# y_pred2 = np.argmax(Y_pred2, axis=1)

# print('Confusion Matrix - Train')
# y_train2 = np.argmax(y_train, axis=1)

# print(confusion_matrix(y_train2, y_pred2))
# print('Classification Report')
# target_names = ['Apr-May', 'Jan-Feb', 'Mar']
# print(classification_report(y_train2, y_pred2, target_names=target_names))